In [19]:
!pip install -U symspellpy

In [20]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data_clean_functions import *

In [21]:
import numpy as np
import pandas as pd

In [22]:
reports = pd.read_csv('../CMV Reports Text.csv')
report_dates = pd.read_csv('../CMV_Report_Dates.csv')

In [23]:

lastrow = pd.DataFrame({'PageNumber': reports.sort_values('PageNumber').iloc[-1]['PageNumber'], 
                                                      'Year': report_dates.iloc[-1]['Year']}, index=[0])

In [24]:

report_dates = pd.concat([report_dates, lastrow]).reset_index(drop=True)

In [25]:
def create_rows_between(report_dates):
    report_between = pd.DataFrame(columns = ['PageNumber', 'Year'])
    years  = []
    pagenums = []
    for i in range(1, len(report_dates)):
        between_n = report_dates['PageNumber'][i] - report_dates['PageNumber'][i-1] -1
        years.extend(np.repeat([report_dates['Year'][i-1]], between_n ))
        
        prev = report_dates['PageNumber'][i-1]
        for k in range(between_n):
            curr = prev + 1
            pagenums.append(curr)
            prev = curr
            
    report_between['Year'] = years 
    report_between['PageNumber'] = pagenums 
    
    return report_between
report_between = create_rows_between(report_dates)


In [26]:
report_dates = pd.concat([report_dates, report_between])
# merge report text with dates
reports = reports.merge(report_dates, how = 'outer').reset_index(drop=True)

In [27]:
reports

,PageNumber,ReportText,IsTableOfContents,IsCoverPage,DocumentBegin,IsAttatchment,Year
0,1,Ne eae |\r\nTHE SECOND ANNUAL REPORT\r\n/ OF T...,0,1,1,0,1996
1,2,is DEPARTMENT OF VETERANS AFFAIRS\r\ny “RY % a...,1,0,0,0,1996
2,3,"i a\r\nStatement of the Chairman, Advisory Com...",1,0,0,0,1996
3,4,A great many people throughout the Congress of...,1,0,0,0,1996
4,5,TABLE OF CONTENTS\r\nStatement of the Chairman...,0,0,0,0,1996
...,...,...,...,...,...,...,...
1011,1012,"Among her many professional honors, Col. Carri...",0,0,0,1,2020
1012,1013,Margaret G. “Muggs” Garvin\r\nAmerican Indian\...,0,0,0,1,2020
1013,1014,"In addition, he served on active duty in the U...",0,0,0,1,2020
1014,1015,**Ginger Miller\r\nAfrican American\r\nGinger ...,0,0,0,1,2020


In [28]:

reports['ReportText'] = [
    remove_extra_space(
        remove_rare_special_character(
            fix_typo(
                remove_lo(
                    remove_vertical_line(
                        fix_roma_numbers(
                            remove_extra_dot(
                                remove_hyphen(
                                    remove_colon(
                                        remove_quotes(
                                            replace_quotes(
                                                remove_bracket(
                                                    normalize_week(
                                                        normalize_month(
                                                            normalize_year(
                                                                normalize_new_lines(
                                                                    i.lower()
                                                                )
                                                            )
                                                        )
                                                    )
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        )
    ).strip() for i in reports['ReportText']]


In [29]:

TOC_idx = [i for i, s in enumerate(reports['ReportText']) if re.findall("TABLE OF CONTENTs?( ?)+(page)?\n".lower(), s ) ]

In [30]:
# there should be 23 "table of contents"
reports_TOC = reports.iloc[TOC_idx]

In [31]:
# adding table of content page for 2016
reports_TOC = pd.concat([reports_TOC, reports[reports['PageNumber'] == 844]]).reset_index(drop=True)
# now we are only missing 2010 and 2019 which is true in the original doc

In [32]:
# add table of content manually
reports_TOC = pd.concat([reports_TOC, pd.DataFrame({'PageNumber':190,
'ReportText':'table of contents\nintroduction\ngeneral overview\nReport on the wilmington, de, meeting\ngeneral\nveterans entrepreneurship\nVeterans Entrepreneurship and Small Business Development\nWorkforce Diversity Plan\nGeneral\nSecretary Principis Comments\nVeterans Benefit Clearinghouse VBC \nVocational Rehabilitation & Employment Service\nServing Native American Veterans\nHepatitis C Treatment\nCenter for Minority Veterans\nCommittee Recommendations\nAppendix A\nAppendix B\nAdvisory Committee Agenda Continued\nAppendix C\n'.lower(),
'IsTableOfContents': 1,
'IsCoverPage': 0,
'DocumentBegin': 1,
'IsAttatchment': 0,
'Year': 2001}, index = [0])]).reset_index(drop=True)


In [33]:
reports_TOC['IsTableOfContents1'] = 1

In [34]:
reports = reports.merge(reports_TOC[['PageNumber', 'IsTableOfContents1']], how = 'left').reset_index(drop=True)

In [35]:
import math
reports['IsTableOfContents1'] = [0 if math.isnan(i) else int(i) for i in reports['IsTableOfContents1']]

In [36]:
articleDF = pd.DataFrame(columns = ['Title', 'ArticleText', 'PageNumber'])
for k in range(len(reports_TOC)):
    articleDF = pd.concat([articleDF, separate_text_into_articles(reports, reports_TOC, k)])

In [38]:
k = 20
articleTitles = clean_table_of_content(reports_TOC, k)
report_i = get_report_one_year(reports, reports_TOC, k)
matchedTitle, pageNum, titleStart = find_title(articleTitles, report_i)
[i for i in articleTitles if i not in matchedTitle] 

['table of contents',
 'subjects page',
 'letter from the advisory committee chai',
 'part executive summa',
 'town hall meeting – houston, texas',
 'page of']

In [39]:
articleTitles


['table of contents',
 'subjects page',
 'letter from the advisory committee chai',
 'part executive summa',
 'summary of year recommendations',
 'acmv year recommendations, rationales, and va responses',
 'agenda year washington, d.c. departmental briefings',
 'agenda houston site visit',
 'town hall meeting – houston, texas',
 'exit briefing with houston regional office, michael e. debakey va medical',
 'and houston national cemetery',
 'agenda year washington, d.c. departmental briefings',
 'action plans follow up',
 'advisory committee biographies',
 'page of']

In [40]:

report_i[report_i['PageNumber'] == 975]['ReportText'].tolist()

['table of contents\nsubjects page\nletter from the advisory committee chai 4\npart executive summa 5\npart ii summary of year recommendations 6\npart iii acmv year recommendations, rationales, and va responses\npart iv agenda year washington, d.c. departmental briefings\npart v agenda houston site visit\npart vi town hall meeting – houston, texas\npart vii exit briefing with houston regional office, michael e. debakey va medical 23\nand houston national cemetery\npart viii agenda year washington, d.c. departmental briefings\nappendix a action plans follow up\nappendix b advisory committee biographies 39\npage 3 of 44']

In [41]:
articleDF = articleDF.reset_index(drop = True)
# put index to column
articleDF = articleDF.reset_index(drop = False)
articleDF = articleDF.rename(columns = {'index': 'ArticleIndex'})


In [42]:

articleDF['Year'] = [reports[reports['PageNumber'] == i[-1]]['Year'].tolist()[0] for i in articleDF['PageNumber']]


In [43]:
recommd = articleDF[['recommendation' in i for i in articleDF['Title']]]


In [44]:
recommd.Year.unique()

array([1996, 1997, 1998, 1999, 2000, 2002, 2004, 2005, 2006, 2007, 2008,
       2009, 2011, 2012, 2013, 2014, 2015, 2017, 2018, 2020, 2016, 2001],
      dtype=int64)

In [45]:
paragraph = []
paragraphNum = []
for i in range(len(recommd)):
    paraghs = recommd.iloc[i]['ArticleText'].split('\n\n')
    paragraph.extend(paraghs)
    paragraphNum.extend(np.repeat(recommd.iloc[i]['ArticleIndex'], len(paraghs)))
paragraphDF = pd.DataFrame({'ParagraphText': paragraph, 'ArticleIndex': paragraphNum})


In [46]:
recommd['ArticleText'] = [remove_new_line(i) for i in recommd['ArticleText']]


C:\Users\VACOFredrD\AppData\Local\Temp\5\ipykernel_21368\1965977422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recommd['ArticleText'] = [remove_new_line(i) for i in recommd['ArticleText']]


In [47]:
recommd = paragraphDF.merge(recommd, how = 'outer')


In [48]:

recommd['ParagraphText'] = [remove_new_line(i) for i in recommd['ParagraphText']]

In [49]:
recommd

,ParagraphText,ArticleIndex,Title,ArticleText,PageNumber,Year
0,"based on the its research, collaboration, part...",8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
1,"2. appropriate 125,000.00 for the fiscal year ...",8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
2,3. recognize the appropnate regional and natio...,8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
3,4. authonze an assessment be conducted and a r...,8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
4,5. establish and fund an intra agency task for...,8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
...,...,...,...,...,...,...
1340,recommendation 1 that the department of vetera...,226,"acmv year recommendations, rationales and va r...",recommendation 1 that the department of vetera...,"[848, 849, 850, 851, 852]",2016
1341,"7 year ncvas minority veterans report, month y...",226,"acmv year recommendations, rationales and va r...",recommendation 1 that the department of vetera...,"[848, 849, 850, 851, 852]",2016
1342,8 https commissiononcare.sites.usa.gov files y...,226,"acmv year recommendations, rationales and va r...",recommendation 1 that the department of vetera...,"[848, 849, 850, 851, 852]",2016
1343,the committee recommends that va number 1. r...,245,committee recommendations,the committee recommends that va number 1. r...,"[201, 202]",2001


In [50]:

recommd_temp = recommd[['ParagraphText','ArticleIndex','ArticleText','Year']]


In [51]:
recommd_temp


,ParagraphText,ArticleIndex,ArticleText,Year
0,"based on the its research, collaboration, part...",8,"based on the its research, collaboration, part...",1996
1,"2. appropriate 125,000.00 for the fiscal year ...",8,"based on the its research, collaboration, part...",1996
2,3. recognize the appropnate regional and natio...,8,"based on the its research, collaboration, part...",1996
3,4. authonze an assessment be conducted and a r...,8,"based on the its research, collaboration, part...",1996
4,5. establish and fund an intra agency task for...,8,"based on the its research, collaboration, part...",1996
...,...,...,...,...
1340,recommendation 1 that the department of vetera...,226,recommendation 1 that the department of vetera...,2016
1341,"7 year ncvas minority veterans report, month y...",226,recommendation 1 that the department of vetera...,2016
1342,8 https commissiononcare.sites.usa.gov files y...,226,recommendation 1 that the department of vetera...,2016
1343,the committee recommends that va number 1. r...,245,the committee recommends that va number 1. r...,2001
